Имеется база данных открытых ключей RSA. Предположим, что исходный ключ симметричного шифрования был зашифрован одним из этих открытых ключей. Чтобы его расшифровать, нам необходимо получить закрытый ключ. Когда в распоряжении есть несколько открытых ключей, можно предположить, что некоторые из них являются слабыми. Это означает, что они имеют общие значения p или q. Узнав разложение n на множители, можно узнать закрытый ключ для данного открытого ключа. Пробуем найти наибольшие общие делители у ключей с помощью алгоритма Евклида. Если НОД каких-либо ключей не равен 1, это означает, что n, соответствующие ключам делятся на НОД. Делим n на НОД и находим искомое разложение.
Найдем слабые ключи:


In [7]:
import rsa
import base64
import math
key_list=[]
for i in range(1, 101):
    path = 'challenge\\'
    path += str(i)
    path += '.pem'
    with open(path, 'rb') as f:
        public_file = f.read()
    pubkey = rsa.PublicKey.load_pkcs1_openssl_pem(public_file)
    key_list.append(pubkey)
    #print(pubkey.n)

for i in range (len(key_list)-1):
    for j in range (i+1,len(key_list)):
        g=math.gcd(key_list[i].n,key_list[j].n)
        if g!=1:
            print(i, j)
            print(g)

6 92
11714776996979588435440066274186746431207255454202665944875994417925671637361979718990753187254963362590523267356636189119746530253241195041886537439669297
8 43
12746114090037338638405558348364753170762024925046410448750084084398232130814610810740410270513256665449277121134270816451924978100353060942631979836542387
28 81
13255021146474500311567608978490890706169542435903223652857277300729054042351345458519493118521383085015716518452176586805584419240894763844777441159376627
33 79
13307692754962545190733618818859249009308397605584526413665519201272482870171338937280605559186730933842250237720203697942551335313430434560173585438909597
57 70
12913750264623462276337425644128711174675813336880465068461018681645080047918835348323621347173714301224881492646558739535118834131362641424915974945262413
59 96
13223199974589313585283471854827363329451685303943205044027865131047695079691108706585803904203301545648101019060082323960067905432343412585098414381558929


Восстановим закрытый ключ, например, для 6 открытого ключа. Значение закрытого ключа получаем путем нахождения обратного к числу $$e$$ по модулю $$φ(n)$$, где $$φ(n) = (p − 1)(q − 1)$$.
$$d=e^(-1) (mod φ(n))$$  
